In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../..")

from heritageconnector import datastore
from heritageconnector.config import field_mapping
from heritageconnector.best_spacy_pipeline import load_model
from heritageconnector.datastore import es, index

import entity_linker

2021-02-12 14:51:04,948 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


### NERLoader

In [10]:
source_description_field = target_description_field = "data.http://www.w3.org/2001/XMLSchema#description"
target_title_field = "graph.@rdfs:label.@value"
target_alias_field = "graph.@skos:altLabel.@value"

record_loader = datastore.RecordLoader("SMG", field_mapping)
ner_loader = datastore.NERLoader(
    record_loader,
    source_es_index = 'heritageconnector',
    target_es_index = 'heritageconnector',
    source_description_field = source_description_field, 
    target_title_field = target_title_field, 
    target_description_field = target_description_field, 
    target_alias_field = target_alias_field,
    entity_types_to_link={"PERSON", "OBJECT", "ORG"}
)

2021-02-12 15:00:51,837 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:00:51,837 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


In [14]:
entity_list = ner_loader.get_list_of_entities_from_es("en_core_web_lg", 500)
len(entity_list), entity_list[0]

2021-02-12 15:11:44,416 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:11:44,416 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:11:44,416 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:11:44,416 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:11:44,498 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-02-12 15:11:44,498 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-02-12 15:11:44,498 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-02-12 15:11:44,498 - heri

0it [00:00, ?it/s]

(1514,
 {'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co151905',
  'item_description': 'Spectacles, extending, brass, English, 1720-1860',
  'ent_label': 'LANGUAGE',
  'ent_text': 'English'})

In [15]:
entity_list_with_links = ner_loader.get_link_candidates(candidates_per_entity_mention=3)
entity_list_with_links[0:10]

2021-02-12 15:11:56,083 - heritageconnector.datastore - INFO - Getting link candidates for each of 1514 entities
2021-02-12 15:11:56,083 - heritageconnector.datastore - INFO - Getting link candidates for each of 1514 entities
2021-02-12 15:11:56,083 - heritageconnector.datastore - INFO - Getting link candidates for each of 1514 entities
2021-02-12 15:11:56,083 - heritageconnector.datastore - INFO - Getting link candidates for each of 1514 entities


  0%|          | 0/1514 [00:00<?, ?it/s]

[{'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co151905',
  'item_description': 'Spectacles, extending, brass, English, 1720-1860',
  'ent_label': 'LANGUAGE',
  'ent_text': 'English'},
 {'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co151905',
  'item_description': 'Spectacles, extending, brass, English, 1720-1860',
  'ent_label': 'DATE',
  'ent_text': '1720-1860'},
 {'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co111946',
  'item_description': '2 cylindrical brown glass bottles with wide necks and screw-threads, no caps, European, 1920-1960',
  'ent_label': 'NORP',
  'ent_text': 'European'},
 {'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co111946',
  'item_description': '2 cylindrical brown glass bottles with wide necks and screw-threads, no caps, European, 1920-1960',
  'ent_label': 'DATE',
  'ent_text': '1920-1960'},
 {'item_uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co17451',
  'item

In [17]:
df = ner_loader.entity_list_as_dataframe

df.head()

2021-02-12 15:23:59,362 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:23:59,362 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:23:59,362 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:23:59,362 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-12 15:23:59,362 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


,item_uri,item_description,ent_label,ent_text,link_candidates
0,https://collection.sciencemuseumgroup.org.uk/o...,"Spectacles, extending, brass, English, 1720-1860",LANGUAGE,English,NaN
1,https://collection.sciencemuseumgroup.org.uk/o...,"Spectacles, extending, brass, English, 1720-1860",DATE,1720-1860,NaN
2,https://collection.sciencemuseumgroup.org.uk/o...,2 cylindrical brown glass bottles with wide ne...,NORP,European,NaN
3,https://collection.sciencemuseumgroup.org.uk/o...,2 cylindrical brown glass bottles with wide ne...,DATE,1920-1960,NaN
4,https://collection.sciencemuseumgroup.org.uk/o...,"Twenty autochromes, mostly of children, by Mrs...",OBJECT,Twenty,[{'uri': 'https://collection.sciencemuseumgrou...


In [33]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
newdf = df.dropna(subset=['link_candidates']).set_index(["item_uri", "item_description", "ent_label", "ent_text"])['link_candidates'].apply(pd.Series).stack().reset_index().rename(columns={"level_4": "candidate_rank"})

candidate_cols = newdf[0].apply(pd.Series)
candidate_cols = candidate_cols.rename(columns={col: f"candidate_{col}" for col in candidate_cols})

pd.concat([newdf, candidate_cols], axis=1).drop(columns=[0])


,item_uri,item_description,ent_label,ent_text,candidate_rank,candidate_uri,candidate_title,candidate_description,candidate_alias
0,https://collection.sciencemuseumgroup.org.uk/objects/co17451,"Twenty autochromes, mostly of children, by Mrs. E. Janet Laing, c. 1907-13. (two cracked).",OBJECT,Twenty,0,https://collection.sciencemuseumgroup.org.uk/objects/co8437527,Louisa's Twenty-First,"A black and white silver gelatin photograph entitled 'Louisa's Twenty-First', taken by Bob Gosani. From the exhibition 'Drum', held at the Impressions Gallery, York between 10 December 1988 and 21 January 1989.",NaN
1,https://collection.sciencemuseumgroup.org.uk/objects/co17451,"Twenty autochromes, mostly of children, by Mrs. E. Janet Laing, c. 1907-13. (two cracked).",OBJECT,Twenty,1,https://collection.sciencemuseumgroup.org.uk/objects/co523867,Twenty-two leather toolbags,Twenty-two leather toolbags,NaN
2,https://collection.sciencemuseumgroup.org.uk/objects/co17451,"Twenty autochromes, mostly of children, by Mrs. E. Janet Laing, c. 1907-13. (two cracked).",OBJECT,Twenty,2,https://collection.sciencemuseumgroup.org.uk/objects/co8088643,Twenty seven quad cable,Twenty seven quad cable with corrugated aluminium sheath and double PVC sheath,NaN
3,https://collection.sciencemuseumgroup.org.uk/objects/co17451,"Twenty autochromes, mostly of children, by Mrs. E. Janet Laing, c. 1907-13. (two cracked).",PERSON,E. Janet Laing,0,https://collection.sciencemuseumgroup.org.uk/objects/co17397,Family album of photographs of Janet Marian Laing (later Mrs,"Family album of photographs of Janet Marian Laing (later Mrs. De Sales La Terriete) as a child, 1898-1903.",NaN
4,https://collection.sciencemuseumgroup.org.uk/objects/co17451,"Twenty autochromes, mostly of children, by Mrs. E. Janet Laing, c. 1907-13. (two cracked).",PERSON,E. Janet Laing,1,https://collection.sciencemuseumgroup.org.uk/objects/co17327,Two hand coloured bromide prints by Janet E,"Two hand coloured bromide prints by Janet E. Laing, c. 1910.",NaN
...,...,...,...,...,...,...,...,...,...
2026,https://collection.sciencemuseumgroup.org.uk/objects/co71408,"Medal, silver, Second Afghan War Medal, with ribbon, awarded to Captain C.F. Gleig, D.A.A. Genl., English, 1880-1881",PERSON,C.F. Gleig,1,https://collection.sciencemuseumgroup.org.uk/objects/co55167,3 cup sensitive anemometer by C.F. C (C.F. Casella,"3 cup sensitive anemometer by C.F. C (C.F. Casella and Co. Ltd), marked `Meteor No4', MO 47/42. Dated 1942",NaN
2027,https://collection.sciencemuseumgroup.org.uk/objects/co71408,"Medal, silver, Second Afghan War Medal, with ribbon, awarded to Captain C.F. Gleig, D.A.A. Genl., English, 1880-1881",PERSON,C.F. Gleig,2,https://collection.sciencemuseumgroup.org.uk/people/cp129279,"Glegg, Donal Lindsay","March House Books, Searching for Ptarmigan Pie, by Alastair Lindsay Glegg, 2007 http://webcache.googleusercontent.com/search?q=cache:44SParcU8Z0J:www.marchhousebooks.com/%3Fpage%3Dshop/disp%26pid%3Dpage_Pie%26CLSN_1948%3D120652414119481442c873234f3ad563+Donal+Lindsay+Glegg&cd=1&hl=en&ct=clnk&gl=uk \n \n Landscape painter and illustrator; worked during the second world war as lecturer on camoflage and concealment for the War Office; began to go blind in 1949 as a result of diabetes, and never painted again.",NaN
2028,https://collection.sciencemuseumgroup.org.uk/objects/co8211036,"Pair of tortoiseshell rimmed spectacles, oval lenses and folding arms with velvet lining, Hudson & Son, Greenwich, London, England, 1850-1905",ORG,Hudson & Son,0,https://collection.sciencemuseumgroup.org.uk/people/cp105146,Hudson & Son,"Clifton, G., Directory of British Scientific Instrument Makers 1550-1851 (Zwemmer, 1995) \n \n Traded at Greenwich, London, England (1850s-1905). Partnership between Frederick T. Hudson and his son, Frederick",NaN
2029,https://collection.sciencemuseumgroup.org.uk/objects/co8211036,"Pair of tortoiseshell rimmed spectacles, oval lenses and folding arms with velvet lining, Hudson & Son, Greenwich, London, Engla

---

In [18]:
nlp = load_model("en_core_web_lg")

2021-02-11 12:13:33,062 - hc_nlp.pipeline - INFO - Loading thesaurus from ../../../heritageconnector/../GITIGNORE_DATA/labels_all_unambiguous_types_people_orgs.jsonl
2021-02-11 12:13:34,427 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 1s


In [28]:
source_description_field = target_description_field = "data.http://www.w3.org/2001/XMLSchema#description"
target_title_field = "graph.@rdfs:label.@value"
target_alias_field = "graph.@skos:altLabel.@value"

es_el = entity_linker.ESEntityLinker(es, index, nlp, source_description_field, target_title_field, target_description_field, target_alias_field)

In [29]:
search_res = es_el._search_es_for_mention("victoria", 100)

search_res[0]

{'query': {'multi_match': {'query': 'victoria', 'fuzziness': 'AUTO', 'fields': ['graph.@rdfs:label.@value', 'graph.@skos:altLabel.@value']}}}


{'uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co25492',
 'data.http://www.w3.org/2001/XMLSchema#description': "Queen Victoria's personal Victoria carriage. Design descends from the French Milard carriage, brought to England by the Prince of Wales, Queen Victoria’s son, for his mother in 1869. The English version was named Victoria in the Queen’s honour and became one of the most popular carriages of the 19th Century. Key feature is its low body and lack of doors for easy access. This was the last carriage to be used by Victoria on a journey to Osborne before her death. It was donated in 1936 by HM King Edward VIII with several other Victorian carriages.",
 'graph.@rdfs:label.@value': "Queen Victoria's 'Victoria' carriage"}

In [32]:
text = search_res[0]['data.http://www.w3.org/2001/XMLSchema#description']
es_el.get_link_candidates_from_spacy_doc(nlp(text), 3)

{'Victoria': [{'uri': 'https://collection.sciencemuseumgroup.org.uk/objects/co25492',
   'data.http://www.w3.org/2001/XMLSchema#description': "Queen Victoria's personal Victoria carriage. Design descends from the French Milard carriage, brought to England by the Prince of Wales, Queen Victoria’s son, for his mother in 1869. The English version was named Victoria in the Queen’s honour and became one of the most popular carriages of the 19th Century. Key feature is its low body and lack of doors for easy access. This was the last carriage to be used by Victoria on a journey to Osborne before her death. It was donated in 1936 by HM King Edward VIII with several other Victorian carriages.",
   'graph.@rdfs:label.@value': "Queen Victoria's 'Victoria' carriage"},
  {'uri': 'https://collection.sciencemuseumgroup.org.uk/people/cp66514',
   'data.http://www.w3.org/2001/XMLSchema#description': 'ODNB: H. C. G. Matthew and K. D. Reynolds, ‘Victoria (1819–1901)’, Oxford Dictionary of National Biogr